# 6 Deployment — Zusammenfassung & Ausblick

## 6.1 Allgemeine Erkenntnisse & Diskussion
Die Evaluation aus Notebook 5 zeigt, dass SHAP und Captum beide valide Erklärungen liefern, sich jedoch in Qualität, Robustheit und Geschwindigkeit deutlich unterscheiden. Der folgende Überblick verdichtet die wichtigsten Resultate und bildet die Grundlage für produktseitige Entscheidungen.

### 6.1.1 Hypothesenkonsolidierung
| Hypothese | Ergebnis | Evidenz (Notebook 5) |
| --- | --- | --- |
| H1: Token-Übereinstimmung | *Teilweise bestätigt* | Top-5-Überlappung Ø 0,54; FN ≈ 0,57, FP ≈ 0,46 |
| H2: Faithfulness | *Bestätigt für SHAP* | SHAP AUC-Ratio ≥ 1,5 in 73 % der Fälle; Captum 49 % |
| H3: Robustheit | *Bestätigt für SHAP, eingeschränkt für Captum* | SHAP ρ ≈ 0,75 stabil, Captum ρ ≈ 0,44 (Füllwörter kritisch) |
| H4: Rechenaufwand | *Bestätigt (Captum schneller)* | SHAP ≈ 22,9 s vs. Captum ≈ 10,0 s (≈ 2,3× schneller) |

### 6.1.2 Kernaussagen für Stakeholder
- **Qualität vs. Kosten:** SHAP liefert konsistentere und robustere Erklärungen, kostet jedoch mehr Laufzeit. Captum eignet sich für schnelle Erstanalysen.
- **Fehlerfokus:** Divergenzen treten besonders bei False Positives auf. Eine manuelle Zweitprüfung dieser Fälle ist empfehlenswert.
- **XAI-Design:** Eine hybride Strategie (Captum für Streaming, SHAP für Audits) verbindet Geschwindigkeit und Aussagekraft.

## 6.2 Überlegungen zum Deployment
Ziel ist ein reproduzierbarer, wartbarer Inferenz-Stack, der sowohl Modellvorhersagen als auch erklärende Artefakte bereitstellt.

### 6.2.1 Deployment-Ziele
- **Hauptziel:** Bereitstellung eines Sentiment-Services mit optionalen XAI-Erklärungen (SHAP für Audits, Captum für Online-Erklärbarkeit).
- **Sekundärziel:** Sicherstellung, dass Ergebnisse aus Notebook 5 reproduzierbar bleiben (Versionierung von Code, Daten, Modellen).

### 6.2.2 Technische Anforderungen
- **Artefakte:** Feinabgestimmtes `vinai/bertweet-base`, Tokenizer, PyTorch-Lightning-Checkpoint, Konfigurationsdateien.
- **Laufzeitumgebung:** Python 3.10+, PyTorch ≥ 2.0, Transformers ≥ 4.57, Captum, SHAP, PyArrow (für Caching), optional ONNX Runtime für Beschleunigung.
- **Hardware:** CPU-Only Betrieb möglich (≈ 100 ms inference pro Tweet), GPU/MPS zur Beschleunigung von Batches und SHAP-Analysen empfehlenswert.
- **Konfigurierbarkeit:** Environment Variablen für Modellpfade, Batchgrößen, Wahl der Erklärmethode.

### 6.2.3 Serving-Architektur
1. **Preprocessing-Service:** Repliziert die Cleaning-Logik aus Notebook 3 (Regex, Normalisierung).
2. **Inference-Service:** Lädt das fine-tuned Modell, bietet REST/gRPC-Endpunkte für Sentiment-Predictions.
3. **Explainability-Service:**
   - *Captum-Pfad:* Schnell, liefert Top-Tokens via LayerIntegratedGradients.
   - *SHAP-Pfad:* Asynchroner Job (Celery/Kafka) für tiefgehende Analysen; Ergebnisse werden gecached.
4. **Storage & Caching:**
   - Redis / SQLite für kurzfristige Ergebnisse.
   - Langfristige Speicherung in S3/Parquet (bereits vorbereitet in Notebook 5).

### 6.2.4 Betriebsprozesse & Überwachung
- **Monitoring:**
  - Metriken: Latenz (p95), Fehlerraten, Anteil der Erklärungsanfragen, Drift-Indikatoren auf Token-Ebene.
  - Logging: Speicherung von (Text, Prediction, Explanation-ID) für Audits (DSGVO-konform).
- **Alerting:** Schwellen für Latenz > 500 ms oder wiederholte Ausfälle beim SHAP-Worker.
- **Governance:** Versionierung über DVC/MLflow; jedes Deployment enthält Modell-, Daten- und Evaluations-Hash.

### 6.2.5 Produktionsreife Checkliste
1. **Reproduzierbares Build:** Dockerfile mit Mehrstufen-Build, automatisierte Tests (PyTest, Smoke Tests).
2. **CI/CD-Pipeline:** GitHub Actions oder Azure DevOps; Stages: Lint → Unit Tests → Modell-Drift-Check → Deployment (staging → prod).
3. **Security:** Secrets-Management, Rate Limiting, Eingabevalidierung.
4. **UX/API:** Konsistentes Schema (JSON) für Antwort inkl. Scores, Token-Attributionslisten, optionaler Visualisierungshyperlink.
5. **Rollbacks:** Canary-Deployments, automatisiertes Zurücksetzen bei Regressionen.

## 6.3 Einschränkungen & Nächste Schritte
Eine reflektierte Betrachtung der Projektgrenzen hilft, Risiken beim Deployment zu adressieren und die Roadmap für weitere Iterationen festzulegen.

### 6.3.1 Bekannte Einschränkungen
- **Datenbasis:** Fokus auf englische Tweets aus 2009; Generalisierbarkeit auf aktuelle Plattformen oder andere Sprachen nicht geprüft.
- **Attributionsstreuung:** Captum-Sensitivität bei Perturbationen deutet auf mögliche Instabilität im Live-Betrieb.
- **Compute-Kosten:** SHAP-Aufwand skaliert schlecht; Produktivbetrieb benötigt dedizierte Ressourcen oder Sampling-Strategien.
- **Bias-Risiko:** Keine Fairness- oder Bias-Audits durchgeführt; vor Deployment erforderlich.

### 6.3.2 Empfohlene nächste Schritte
1. **Erweiterte Evaluation:** Größere Stichprobe (> 200 Tweets), zusätzliche Perturbationen (Code-Switching, Emojis).
2. **User Research:** Interviews mit Analyst:innen zur Verständlichkeit der Erklärungen, Anpassung des UI basierend auf Feedback.
3. **Hybrid-Explainability:** Implementierung eines zweistufigen Systems (Captum online, SHAP offline) und A/B-Test gegen rein Captum-basierten Flow.
4. **Model Monitoring MVP:** Aufbau eines Monitoring-Dashboards (Grafana/Prometheus) mit Echtzeit-Alarmierung.
5. **Compliance-Check:** Datenschutz-Freigabe, Dokumentation für Audit (Modellkarten, Datenblätter).

## 6.4 Roadmap zur Produktionsreife
- **Kurzfristig (0–1 Monat):** Dockerisieren, CI/CD aufsetzen, Captum-Only Service ausrollen, Monitoring-Basics implementieren.
- **Mittelfristig (1–3 Monate):** SHAP-Offloading-Service etablieren, UI-Komponenten bauen, Fairness-Checks durchführen.
- **Langfristig (>3 Monate):** Erweiterung auf Multilingualität, kontinuierliches Retraining mit Feedback-Schleifen, Integration in CRM/Support-Systeme.

## 6.5 Production-Ready FastAPI Implementation

Below is a complete FastAPI service implementation incorporating all deployment best practices identified in previous sections.

In [ ]:
"""
Production-Ready FastAPI Sentiment Analysis Service with XAI
Save this as: api/main.py
Run with: uvicorn main:app --host 0.0.0.0 --port 8000
"""

from fastapi import FastAPI, HTTPException, status
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict, Literal
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from captum.attr import LayerIntegratedGradients
import shap
import re
import time
import logging
from pathlib import Path
from prometheus_client import Counter, Histogram, generate_latest
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Prometheus metrics
PREDICTION_COUNTER = Counter('predictions_total', 'Total predictions made', ['sentiment'])
PREDICTION_LATENCY = Histogram('prediction_latency_seconds', 'Prediction latency')
EXPLANATION_COUNTER = Counter('explanations_total', 'Total explanations generated', ['method'])
EXPLANATION_LATENCY = Histogram('explanation_latency_seconds', 'Explanation latency', ['method'])

# Initialize FastAPI app
app = FastAPI(
    title="BERTweet Sentiment Analysis API",
    description="Production-ready sentiment analysis with SHAP/Captum explainability",
    version="1.0.0"
)

# Global model variables (loaded at startup)
model = None
tokenizer = None
lig_explainer = None
shap_explainer = None
device = None

# Configuration
MODEL_PATH = Path("../models/bertweet_sentiment_finetuned")
MAX_LENGTH = 128
BATCH_SIZE = 32


class PredictionRequest(BaseModel):
    """Request schema for sentiment prediction"""
    text: str = Field(..., min_length=1, max_length=500, description="Tweet text to analyze")
    explain: bool = Field(False, description="Whether to include attribution explanations")
    explain_method: Literal["captum", "shap", "both"] = Field("captum", description="Explanation method")
    top_k: int = Field(5, ge=1, le=20, description="Number of top attribution tokens to return")
    
    @validator('text')
    def clean_text(cls, v):
        """Apply basic text cleaning"""
        v = v.strip()
        if not v:
            raise ValueError("Text cannot be empty after cleaning")
        return v


class TokenAttribution(BaseModel):
    """Token-level attribution score"""
    token: str
    attribution: float
    position: int


class PredictionResponse(BaseModel):
    """Response schema for predictions"""
    sentiment: Literal["negative", "positive"]
    confidence: float = Field(..., ge=0.0, le=1.0)
    probabilities: Dict[str, float]
    latency_ms: float
    explanations: Optional[Dict[str, List[TokenAttribution]]] = None


def clean_tweet(text: str) -> str:
    """Replicate cleaning logic from Notebook 3"""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    return re.sub(r'\s+', ' ', text).strip()


@app.on_event("startup")
async def load_model():
    """Load model and explainers at startup"""
    global model, tokenizer, lig_explainer, shap_explainer, device
    
    logger.info("Loading model and tokenizer...")
    if not MODEL_PATH.exists():
        raise RuntimeError(f"Model not found at {MODEL_PATH}")
    
    # Device selection
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    
    logger.info(f"Using device: {device}")
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
    
    # Initialize Captum explainer
    logger.info("Initializing Captum LayerIntegratedGradients...")
    lig_explainer = LayerIntegratedGradients(
        lambda input_ids, attention_mask: model(input_ids=input_ids, attention_mask=attention_mask).logits,
        model.roberta.embeddings
    )
    
    # Initialize SHAP explainer
    logger.info("Initializing SHAP explainer...")
    def shap_predict_fn(texts):
        encodings = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LENGTH, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**encodings).logits
        return torch.softmax(logits, dim=1).cpu().numpy()
    
    shap_explainer = shap.Explainer(shap_predict_fn, tokenizer)
    
    logger.info("Model and explainers loaded successfully")


@app.get("/health", status_code=status.HTTP_200_OK)
async def health_check():
    """Health check endpoint for monitoring"""
    if model is None or tokenizer is None:
        raise HTTPException(
            status_code=status.HTTP_503_SERVICE_UNAVAILABLE,
            detail="Model not loaded"
        )
    return {
        "status": "healthy",
        "model_loaded": model is not None,
        "device": str(device)
    }


@app.get("/metrics")
async def metrics():
    """Prometheus metrics endpoint"""
    return generate_latest()


@app.post("/predict", response_model=PredictionResponse, status_code=status.HTTP_200_OK)
async def predict_sentiment(request: PredictionRequest):
    """
    Predict sentiment with optional explainability
    
    - **text**: Tweet text to analyze (1-500 characters)
    - **explain**: Whether to include token attributions
    - **explain_method**: Explanation method (captum, shap, or both)
    - **top_k**: Number of top attribution tokens to return
    """
    start_time = time.time()
    
    try:
        # Preprocess text
        cleaned_text = clean_tweet(request.text)
        if not cleaned_text:
            raise HTTPException(
                status_code=status.HTTP_400_BAD_REQUEST,
                detail="Text is empty after cleaning"
            )
        
        # Tokenize and predict
        with PREDICTION_LATENCY.time():
            encoding = tokenizer(
                cleaned_text,
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(device)
            
            with torch.no_grad():
                logits = model(**encoding).logits
                probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        
        sentiment_idx = int(probs.argmax())
        sentiment = "positive" if sentiment_idx == 1 else "negative"
        confidence = float(probs[sentiment_idx])
        
        # Update metrics
        PREDICTION_COUNTER.labels(sentiment=sentiment).inc()
        
        # Generate explanations if requested
        explanations = None
        if request.explain:
            explanations = {}
            
            if request.explain_method in ["captum", "both"]:
                with EXPLANATION_LATENCY.labels(method="captum").time():
                    captum_attrs = generate_captum_attributions(
                        cleaned_text, encoding, sentiment_idx, request.top_k
                    )
                    explanations["captum"] = captum_attrs
                    EXPLANATION_COUNTER.labels(method="captum").inc()
            
            if request.explain_method in ["shap", "both"]:
                with EXPLANATION_LATENCY.labels(method="shap").time():
                    shap_attrs = generate_shap_attributions(
                        cleaned_text, sentiment_idx, request.top_k
                    )
                    explanations["shap"] = shap_attrs
                    EXPLANATION_COUNTER.labels(method="shap").inc()
        
        latency_ms = (time.time() - start_time) * 1000
        
        return PredictionResponse(
            sentiment=sentiment,
            confidence=confidence,
            probabilities={"negative": float(probs[0]), "positive": float(probs[1])},
            latency_ms=latency_ms,
            explanations=explanations
        )
        
    except Exception as e:
        logger.error(f"Prediction error: {str(e)}")
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail=f"Prediction failed: {str(e)}"
        )


def generate_captum_attributions(text: str, encoding, target_class: int, top_k: int) -> List[TokenAttribution]:
    """Generate Captum LayerIntegratedGradients attributions"""
    baseline = torch.zeros_like(encoding['input_ids'])
    
    attributions = lig_explainer.attribute(
        inputs=encoding['input_ids'],
        baselines=baseline,
        additional_forward_args=(encoding['attention_mask'],),
        target=target_class,
        return_convergence_delta=False
    )
    
    # Sum over embedding dimension
    token_attrs = attributions.sum(dim=-1).squeeze().cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze().cpu().tolist())
    
    # Filter special tokens and get top-k
    results = []
    for idx, (token, attr) in enumerate(zip(tokens, token_attrs)):
        if token not in tokenizer.all_special_tokens:
            results.append(TokenAttribution(token=token, attribution=float(attr), position=idx))
    
    results.sort(key=lambda x: abs(x.attribution), reverse=True)
    return results[:top_k]


def generate_shap_attributions(text: str, target_class: int, top_k: int) -> List[TokenAttribution]:
    """Generate SHAP Partition explainer attributions"""
    shap_values = shap_explainer([text])
    
    # Extract token-level SHAP values for target class
    tokens = shap_values.data[0]
    values = shap_values.values[0, :, target_class]
    
    results = []
    for idx, (token, value) in enumerate(zip(tokens, values)):
        if token.strip():  # Non-empty tokens
            results.append(TokenAttribution(token=token, attribution=float(value), position=idx))
    
    results.sort(key=lambda x: abs(x.attribution), reverse=True)
    return results[:top_k]


@app.post("/batch_predict", status_code=status.HTTP_200_OK)
async def batch_predict(texts: List[str], explain: bool = False):
    """Batch prediction endpoint for multiple texts"""
    if len(texts) > BATCH_SIZE:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=f"Batch size exceeds maximum of {BATCH_SIZE}"
        )
    
    results = []
    for text in texts:
        request = PredictionRequest(text=text, explain=explain)
        result = await predict_sentiment(request)
        results.append(result)
    
    return {"predictions": results, "count": len(results)}


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

### 6.5.1 API Usage Examples

Example requests using curl or Python:

In [ ]:
# Example 1: Simple prediction without explanations
import requests

response = requests.post(
    "http://localhost:8000/predict",
    json={"text": "I love this product! Amazing quality!"}
)
print(response.json())
# Output: {"sentiment": "positive", "confidence": 0.95, "probabilities": {...}, "latency_ms": 45.2}

# Example 2: Prediction with Captum explanations
response = requests.post(
    "http://localhost:8000/predict",
    json={
        "text": "Terrible service, very disappointed",
        "explain": True,
        "explain_method": "captum",
        "top_k": 3
    }
)
print(response.json())

# Example 3: Batch prediction
response = requests.post(
    "http://localhost:8000/batch_predict",
    json={
        "texts": [
            "Great experience!",
            "Worst purchase ever",
            "It's okay, nothing special"
        ],
        "explain": False
    }
)
print(response.json())

# Example 4: Health check
response = requests.get("http://localhost:8000/health")
print(response.json())
# Output: {"status": "healthy", "model_loaded": true, "device": "cpu"}

### 6.5.2 Deployment Requirements

To deploy this API, install additional dependencies:

In [ ]:
# Install FastAPI and production dependencies
pip install fastapi uvicorn[standard] pydantic prometheus-client

# Add to requirements.txt:
# fastapi==0.109.0
# uvicorn[standard]==0.27.0
# pydantic==2.5.0
# prometheus-client==0.19.0

# Run the API locally
uvicorn main:app --host 0.0.0.0 --port 8000 --reload

# For production deployment with Gunicorn:
gunicorn -w 4 -k uvicorn.workers.UvicornWorker main:app --bind 0.0.0.0:8000